# Subthreshold Slope

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
csv_path = './data/subthreshold_slope.csv'
data = pd.read_csv(csv_path)

vgs = data['VGS']
M0_100 = data['leafValue( IS("/M1/D") "temperature" 100 )']
M2_100 = data['leafValue( IS("/M2/D") "temperature" 100 )']
M0_27 = data['leafValue( IS("/M1/D") "temperature" 27 )']
M2_27 = data['leafValue( IS("/M2/D") "temperature" 27 )']

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
axs[0].plot(vgs, M0_27, label='27C', color='black', linewidth=2)
axs[0].plot(vgs, M0_100, label='100C', color='red', linestyle='--', linewidth=2)
axs[0].set_yscale('log')
axs[0].set_title('M0: 200n/60n')

axs[1].plot(vgs, M2_27, label='27C', color='black', linewidth=2)
axs[1].plot(vgs, M2_100, label='100C', color='red', linestyle='--', linewidth=2)
axs[1].set_yscale('log')
axs[1].set_title('M2: 200n/100n')

# Perform exponential regression for each dataset without sklearn
def perform_exponential_regression(vgs, data):
    mask = (vgs >= 0.0) & (vgs <= 0.3)
    vgs_filtered = vgs[mask]
    data_filtered = data[mask]
    log_data_filtered = np.log(data_filtered)
    A = np.vstack([vgs_filtered, np.ones(len(vgs_filtered))]).T
    slope, intercept = np.linalg.lstsq(A, log_data_filtered, rcond=None)[0]
    return slope, intercept

# Exponential regression for M0_27
slope_M0_27, intercept_M0_27 = perform_exponential_regression(vgs, M0_27)
axs[0].plot(vgs, np.exp(slope_M0_27 * vgs + intercept_M0_27), linestyle=':', color='blue')

# Exponential regression for M0_100
slope_M0_100, intercept_M0_100 = perform_exponential_regression(vgs, M0_100)
axs[0].plot(vgs, np.exp(slope_M0_100 * vgs + intercept_M0_100), linestyle=':', color='green')

# Exponential regression for M2_27
slope_M2_27, intercept_M2_27 = perform_exponential_regression(vgs, M2_27)
axs[1].plot(vgs, np.exp(slope_M2_27 * vgs + intercept_M2_27), linestyle=':', color='blue')

# Exponential regression for M2_100
slope_M2_100, intercept_M2_100 = perform_exponential_regression(vgs, M2_100)
axs[1].plot(vgs, np.exp(slope_M2_100 * vgs + intercept_M2_100), linestyle=':', color='green')

axs[0].set_ylim(1e-10, 1e-1)
axs[1].set_ylim(1e-10, 1e-1)

axs[0].legend()
axs[1].legend()

# Calculate the slope of the exponential function on a log scale
def calculate_slope_per_decade(slope):
    return 1 / slope

# Calculate slopes per decade for each dataset
slope_per_decade_M0_27 = calculate_slope_per_decade(slope_M0_27)
slope_per_decade_M0_100 = calculate_slope_per_decade(slope_M0_100)
slope_per_decade_M2_27 = calculate_slope_per_decade(slope_M2_27)
slope_per_decade_M2_100 = calculate_slope_per_decade(slope_M2_100)

print(f"Slope per decade for M0 at 27C: {slope_per_decade_M0_27:.2e} V/decade")
print(f"Slope per decade for M0 at 100C: {slope_per_decade_M0_100:.2e} V/decade")
print(f"Slope per decade for M2 at 27C: {slope_per_decade_M2_27:.2e} V/decade")
print(f"Slope per decade for M2 at 100C: {slope_per_decade_M2_100:.2e} V/decade")

plt.show()